In [15]:
import pandas as pd
import datetime
from pandasql import sqldf
import sqlite3
import plotly.express as px

pd.options.display.max_rows = 50
pd.options.display.max_columns = 100

mysqldf = lambda q: sqldf(q, globals())

In [10]:
conn = sqlite3.connect('optionsQuotes1.db')
#c = conn.cursor()
df = pd.read_sql('select * from data',conn)
conn.close()
df['time_converted'] = pd.to_datetime(df['time_converted'])

In [50]:
max_min_df = mysqldf("""
        
        WITH 
        
        MAX_TABLE AS (      
        select distinct date(time_converted) as date
        ,day_name
        , DTE_adjusted
        , day_classification
        , max(options_pct_change) as max_options_pct_change
        , time_converted as time_of_max_change
         from df 
         where options_earliest_open>0.15
        group by date(time_converted), dte
        having max_options_pct_change = options_pct_change
        ), 
        
        MIN_TABLE AS (
        select distinct date(time_converted) as date
        , DTE_adjusted
        , day_classification
        , min(options_pct_change) as min_options_pct_change
        , time_converted as time_of_min_change
         from df 
         where options_earliest_open>0.15
        group by date(time_converted), dte
        having min_options_pct_change = options_pct_change
        )
        
        select 
        MAX_TABLE.date, MAX_TABLE.day_name, MAX_TABLE.DTE_adjusted, 
        MAX_TABLE.day_classification,MAX_TABLE.max_options_pct_change, MAX_TABLE.time_of_max_change
        , MIN_TABLE.min_options_pct_change, MIN_TABLE.time_of_min_change 
        from MAX_TABLE
        LEFT JOIN MIN_TABLE ON
        MAX_TABLE.date = MIN_TABLE.date 
        AND MAX_TABLE.DTE_adjusted = MIN_TABLE.DTE_adjusted
        """) 

In [54]:
df_sql = mysqldf("""SELECT
  DATE(Time_Converted) AS 'Date',DTE_ADJUSTED,
  MAX(CASE WHEN TYPE = 'call' THEN options_pct_change ELSE NULL END) AS 'Call_Max_Daily',
  MAX(CASE WHEN TYPE = 'put' THEN options_pct_change ELSE NULL END) AS 'Put_Max_Daily',
  MIN(CASE WHEN TYPE = 'call' THEN options_pct_change ELSE NULL END) AS 'Call_Min_Daily',
  MIN(CASE WHEN TYPE = 'put' THEN options_pct_change ELSE NULL END) AS 'Put_Min_Daily'
FROM df
WHERE options_earliest_open>0.15
GROUP BY DATE(Time_Converted), DTE_ADJUSTED
""")

In [55]:
df_sql

,Date,DTE_adjusted,Call_Max_Daily,Put_Max_Daily,Call_Min_Daily,Put_Min_Daily
0,2023-01-03,0,1.000000,11.937500,0.003460,1.000000
1,2023-01-03,1,1.000000,5.588235,0.076923,0.913652
2,2023-01-03,2,1.053448,4.850000,0.147541,0.948258
3,2023-01-04,0,2.705882,1.419355,0.011765,0.014706
4,2023-01-04,1,2.100000,3.694444,0.150000,0.173913
...,...,...,...,...,...,...
654,2023-11-28,1,4.066667,1.080241,0.906860,0.185185
655,2023-11-28,2,3.739130,1.134094,0.875283,0.305085
656,2023-11-29,0,1.500000,6.826087,0.008929,0.652174
657,2023-11-29,1,1.321429,3.346154,0.071429,0.800000


In [71]:
# some charting stuff
fig = px.line(df_sql[df_sql['DTE_adjusted'] ==0], 
              x="Date", y=["Put_Max_Daily","Call_Max_Daily"], markers=True)
fig.update_traces(line=dict(width=0.0))
fig.show()

In [72]:
def classi

,index,volume_options,volume_weighted_options,open_options,close_options,high_options,low_options,timestamp_options,number of trades_options,ticker,full_name,type,strike,expiry,equity_start_price,time_converted,open_equity,close_equity,high_equity,low_equity,timestamp_equity,number of trades_equity,_merge,equity_pct_change,options_pct_change,options_earliest_open,file_source,equity_pct_change_normalized,day_classification,DTE,day_name,DTE_adjusted
0,0,4,10.8500,11.53,10.45,11.53,10.45,1672757100000,4,SPY,O:SPY230103C00373000,call,373,2023-01-03 00:00:00,385.29,2023-01-03 09:45:00,385.290,381.7100,385.400,381.6601,1.672757e+12,35250.0,both,1.000000,1.000000,11.53,0-2DTE_spy_options_01Jan23-28Feb23v2.pkl,0.000000,high,0,Tuesday,0
1,1,14,8.9964,9.25,8.50,9.25,8.50,1672758000000,6,SPY,O:SPY230103C00373000,call,373,2023-01-03 00:00:00,385.29,2023-01-03 10:00:00,381.770,381.1650,382.580,380.9300,1.672758e+12,27992.0,both,0.990864,0.802255,11.53,0-2DTE_spy_options_01Jan23-28Feb23v2.pkl,-0.913598,high,0,Tuesday,0
2,2,41,7.3554,7.31,7.82,7.82,7.29,1672758900000,5,SPY,O:SPY230103C00373000,call,373,2023-01-03 00:00:00,385.29,2023-01-03 10:15:00,381.180,380.9400,381.370,380.1450,1.672759e+12,24872.0,both,0.989333,0.633998,11.53,0-2DTE_spy_options_01Jan23-28Feb23v2.pkl,-1.066729,high,0,Tuesday,0
3,3,19,7.7484,7.44,7.90,7.90,7.44,1672759800000,6,SPY,O:SPY230103C00373000,call,373,2023-01-03 00:00:00,385.29,2023-01-03 10:30:00,380.890,381.0600,381.390,380.0700,1.672760e+12,24431.0,both,0.988580,0.645273,11.53,0-2DTE_spy_options_01Jan23-28Feb23v2.pkl,-1.141997,high,0,Tuesday,0
4,4,12,6.8425,6.73,6.89,6.98,6.67,1672762500000,7,SPY,O:SPY230103C00373000,call,373,2023-01-03 00:00:00,385.29,2023-01-03 11:15:00,379.660,380.0382,380.460,379.4800,1.672762e+12,25032.0,both,0.985388,0.583695,11.53,0-2DTE_spy_options_01Jan23-28Feb23v2.pkl,-1.461237,high,0,Tuesday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718710,718710,60,14.8070,14.78,14.81,14.81,14.78,1701287100000,2,SPY,O:SPY231201P00470000,put,470,2023-12-01 00:00:00,457.60,2023-11-29 14:45:00,455.230,455.5250,455.560,455.1900,1.701287e+12,10229.0,both,0.994821,1.052707,14.04,0-2DTE_spy_options_01Sep23-31Dec23.pkl,-0.517920,average,2,Wednesday,2
718711,718711,3,0.0167,0.02,0.01,0.02,0.01,1701271800000,2,SPY,O:SPY231201C00471000,call,471,2023-12-01 00:00:00,457.60,2023-11-29 10:30:00,456.480,456.4100,456.650,456.1100,1.701272e+12,17957.0,both,0.997552,1.000000,0.02,0-2DTE_spy_options_01Sep23-31Dec23.pkl,-0.244755,average,2,Wednesday,2
718712,718712,10,0.0100,0.01,0.01,0.01,0.01,1701282600000,1,SPY,O:SPY231201C00471000,call,471,2023-12-01 00:00:00,457.60,2023-11-29 13:30:00,456.295,455.9999,456.310,455.6950,1.701283e+12,9800.0,both,0.997148,0.500000,0.02,0-2DTE_spy_options_01Sep23-31Dec23.pkl,-0.285184,average,2,Wednesday,2
718713,718713,10,0.0100,0.01,0.01,0.01,0.01,1701285300000,1,SPY,O:SPY231201C00471000,call,471,2023-12-01 00:00:00,457.60,2023-11-29 14:15:00,456.170,455.8850,456.265,455.8500,1.701285e+12,9116.0,both,0.996875,0.500000,0.02,0-2DTE_spy_options_01Sep23-31Dec23.pkl,-0.312500,average,2,Wednesday,2


In [58]:
df_1 = px.data.stocks()
df_1

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472
